# Introduction to PyTorch

**OBJECTIVES**

- Implement basic linear regression with `pytorch`
- Implement binary classification with `pytorch`
- Use transfer learning with `pytorch`

### `pytorch` tensors

In [ ]:
# !pip install torch -U

In [ ]:
import torch
import torch.nn as nn

In [ ]:
#create a weight and bias vector


In [ ]:
#look at the weight


In [ ]:
#feed it forward


In [ ]:
#create input


In [ ]:
#evaluate


### The Simple Linear Model

In [ ]:
#set random seed

#basic linear model

#print the model


In [ ]:
#input

#pass to model


Now, we will create a class that inherits from the `nn.Module` and will implement our regression forward pass.

In [ ]:
#linear regression class


In [ ]:
#seed and pass input 2, 1


In [ ]:
#look at the parameters


In [ ]:
#create an input


In [ ]:
#feed it forward


### Synthetic Dataset

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
#create X and y
X = torch.randn(100, 1)*10
y = X + torch.randn(100, 1)*3

In [ ]:
#scatter


In [ ]:
#look at our parameters


In [ ]:
#save as w, b


In [ ]:
#function to get parameters


In [ ]:
#function for plotting fit


In [ ]:
#plot it


### Improving the Fit

$$(y - (wx + b))^2$$

Choose $w$ to minimize error.

$$w_1 = w_0 - \alpha f'(w)$$

In [ ]:
#initial weight and intercept


In [ ]:
#compute the loss for (x = -3)


In [ ]:
#examine it


In [ ]:
#derivative of loss


In [ ]:
#examine it


In [ ]:
#update weight


Typically we also have bias term and update according to gradients.

$$m_1 = m_0 - \alpha f'(m)$$
$$b_1 = b_0 - \alpha f'(b)$$

### Implementing in PyTorch

In [ ]:
#class for regression


In [ ]:
#seed and make data


In [ ]:
#instantiate the model


In [ ]:
#set the loss


In [ ]:
#set the optimizer


In [ ]:
#train it

for i in range(epochs):
    #feed data forward
    
    #compute the loss
    

In [ ]:
#plot the losses


In [ ]:
#examine the fit


In [ ]:
#weight


In [ ]:
#bias


### Using with Data

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
sscaler = StandardScaler()

### Classification

In [ ]:
from sklearn.datasets import load_breast_cancer

In [ ]:
cancer = load_breast_cancer()

In [ ]:
X = cancer.data
y = cancer.target

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
sscaler = StandardScaler()

In [ ]:
X = sscaler.fit_transform(X)

In [ ]:
#classifier class


In [ ]:
#instantiate it


In [ ]:
#move from numpy


In [ ]:
#function for training


In [ ]:
#reshaping y


In [ ]:
#optimizer


In [ ]:
#train it


In [ ]:
#make predictions


In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
#look at confusion matrix


### Pretrained Network

In [ ]:
from torchvision import models

In [ ]:
#models available
dir(models)

In [ ]:
from PIL import Image

In [ ]:
#read in using PIL
dog = Image.open('data/len_2.jpg')

In [ ]:
#take a peek
plt.imshow(dog)
plt.title('Lenny')
plt.axis('off');

In [ ]:
#bring in resnet
resnet = models.resnet101(pretrained = True)
#resnet.eval()

In [ ]:
from torchvision import transforms

In [ ]:
#preprocess image
preprocess = transforms.Compose([transforms.ToTensor(),
                                 transforms.Normalize(
                                 mean = [0.485, .456, .406],
                                 std=[0.229, 0.224, 0.225])])


In [ ]:
#preprocess the dog
lenny = preprocess(dog)
#lenny2 = preprocess(dog2)

In [ ]:
#dimension fix
batch_lenny = torch.unsqueeze(lenny, 0)

In [ ]:
#pass in image
out = resnet(batch_lenny)

In [ ]:
#labels of imagenet classes
with open('data/imagenet_classes.txt') as f:
    labels = [line.strip() for line in f.readlines()]

In [ ]:
#highest probability
_, index = torch.max(out, 1)

In [ ]:
#change output to percent
percent = torch.nn.functional.softmax(out, dim=1)[0]*100
labels[index[0]], percent[index[0]].item()

In [ ]:
_, indices = torch.sort(out, descending = True)
[(labels[idx], percent[idx].item()) for idx in indices[0][:10]]

### `deeplabv2_resnet101`

A second model that performs semantic segmentation.  Here, we aim at each pixel and use the pretrained classes to identify whether pixels are part of these categories. 

In [ ]:
model = torch.hub.load('pytorch/vision:v0.6.0', 'deeplabv3_resnet101', pretrained=True)

In [ ]:
#set to evaluation mode so channels are not an issue
model.eval()
with torch.no_grad():
    outtie = model(batch_lenny)

In [ ]:
outtie.keys()

In [ ]:
output_preds = outtie['out'][0].argmax(0)

In [ ]:
palette = torch.tensor([2 ** 25 - 1, 2 ** 15 - 1, 2 ** 21 - 1])
colors = torch.as_tensor([i for i in range(21)])[:, None] * palette
colors = (colors % 255).numpy().astype("uint8")

In [ ]:
# plot the semantic segmentation predictions of 21 classes in each color
lenny_segments = Image.fromarray(output_preds.byte().cpu().numpy()).resize(dog.size)
lenny_segments.putpalette(colors)

In [ ]:
plt.imshow(lenny_segments)
plt.axis('off');
plt.title('Bucket Head Lenny');

### More pretrained models

- [Torchvision models: Semantic Segmentation](https://pytorch.org/vision/0.8/models.html#semantic-segmentation)

- [Basic Convnet with Pytorch](https://pytorch.org/tutorials/recipes/recipes/defining_a_neural_network.html)